In [1]:
%run load_data.ipynb
print (dir())

{0: 3356849, 1: 47, 2: 159, 4: 885653, 67108864: 93686, 134217728: 28959}
{0: 4290039, 1: 45, 2: 349, 4: 1627, 67108864: 16092, 134217728: 29906}
0
1
10
100
100000000000000000000000000
1000000000000000000000000000


KeyboardInterrupt: 

['DataFrame', 'In', 'Math', 'Out', 'Series', 'Table', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'bottleneck', 'display', 'exit', 'fits', 'get_df_list', 'get_flag_counts', 'get_ipython', 'h5py', 'load_mcal', 'load_true', 'mcal2', 'mcal3', 'mcal4', 'mcal5', 'np', 'number', 'numexpr', 'pd', 'plt', 'quit', 'tqdm']


In [2]:
import healpy as hp
import astropy_healpix as a_hp
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# Choose NSIDE

In [3]:
NSIDE=256 #or 512
NPIX=hp.nside2npix(NSIDE)
print ("The number of pixels is: {}.".format(NPIX))
G_INDEX=np.arange(NPIX)

The number of pixels is: 786432.


In [4]:
final_df_list=[None]*4
for i,catalog in enumerate(mcal_df_list_positive):
    final_df_list[i]=pd.concat([catalog,photdf[['true_cm_g_1','true_cm_g_2']]],axis=1,join='inner')

NameError: name 'mcal_df_list_positive' is not defined

In [ ]:
def get_healpix(df):
    print ("The length of the df is: {}.".format(len(df)))
    print ("The ratio of the length of the catagory to the total pixel number is: {:.2%}".format(len(df)/NPIX))
    df['ipix']=df.apply(lambda x:hp.ang2pix(NSIDE,np.pi/2-np.deg2rad(x['dec']),np.deg2rad(x['ra']),nest=True),axis=1)
    print("The propotion of rendered objects is: {:.2%}".format(np.sum(df['ipix']!=0)/len(df)))
    print ("Resetting indexes")
    df_ipix=df.set_index('ipix')
    df_ipix_sorted=df_ipix.sort_index()
    return (df_ipix_sorted)

def group_ipix(df_ipix_sorted):
    number_mask=df_ipix_sorted.groupby('ipix').count()['R11']<10
    df_sorted_grouped=df_ipix_sorted.groupby('ipix').mean()
    df_sorted_grouped_masked=df_sorted_grouped[number_mask]
    print ("The mean of R11 is {}, max is {}, min is {}".format(df_sorted_grouped_masked['R11'].mean(),df_sorted_grouped_masked['R11'].max(),df_sorted_grouped_masked['R11'].min()))
    print ("The mean of R22 is {}, max is {}, min is {}".format(df_sorted_grouped_masked['R22'].mean(),df_sorted_grouped_masked['R22'].max(),df_sorted_grouped_masked['R22'].min()))
    R11 = df_sorted_grouped_masked['R11'].mean()
    R22 = df_sorted_grouped_masked['R22'].mean()
    df_sorted_grouped_masked['calculated_g_1']=df_sorted_grouped_masked.apply(lambda x:x['e_1']/R11,axis=1)
    df_sorted_grouped_masked['calculated_g_2']=df_sorted_grouped_masked.apply(lambda x:x['e_2']/R22,axis=1)
    df_sorted_grouped_masked['delta_g1']=df_sorted_grouped_masked.apply(lambda x:x['calculated_g_1']-x['true_cm_g_1'],axis=1)
    df_sorted_grouped_masked['delta_g2']=df_sorted_grouped_masked.apply(lambda x:x['calculated_g_2']-x['true_cm_g_2'],axis=1)
#     print ("The propotion of averaged objects is: {:.2%}".format((len(df_ipix_sorted)-len(df_sorted_grouped_masked))/len(df_ipix_sorted)))
    print ("---------------------------------")
    return (df_sorted_grouped_masked)

In [ ]:
ipix_sorted = [get_healpix(df) for df in final_df_list]

In [ ]:
ipix_sorted_filtered_grouped=[group_ipix(df) for df in ipix_sorted]

In [ ]:
def fill_for_plot(df,key_array):
    df_filled=df[key_array].reindex(index=G_INDEX,fill_value=np.nan)
    print("The propotion of filled pixels is: {}.".format(np.sum(df_filled['delta_g1']>0)/NPIX))
    return(df_filled)

In [ ]:
hp_filled_list=[fill_for_plot(df,['delta_g1','delta_g2']) for df in ipix_sorted_filtered_grouped]

In [ ]:
fig1,(ax1,ax2)=plt.subplots(ncols=2)

plt.axes(ax1)
hp.mollview(hp_filled_list[0]['delta_g1'].to_numpy(),nest=True,title="$\delta g_1$ for riz /w MOF",hold=True)
hp.graticule()

plt.axes(ax2)
hp.mollview(hp_filled_list[1]['delta_g1'].to_numpy(),nest=True,title="$\delta g_1$ for riz /wo MOF",hold=True)
hp.graticule()



plt.title('HEALPix comparison')

plt.savefig('/home/cz136/project/balrog/image/hp/sample_delta_g1.png',dpi=2048)
plt.close()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)

plt.axes(ax1)
hp.mollview(np.random.random(hp.nside2npix(32)), hold=True)

plt.axes(ax2)
hp.mollview(np.arange(hp.nside2npix(32)), hold=True)

plt.close()

# Compare maps

In [ ]:
maps=h5py.File('/home/cz136/project/balrog/data/Y3_GOLD_2_2_1_maps.h5','r')['maps'] #no neighbor

In [ ]:
gold=h5py.File('/home/cz136/project/balrog/data/Y3_GOLD_2_2_1_maps.h5','r')

In [ ]:
list(gold.keys())

In [ ]:
list(gold['maps'].keys())

In [ ]:
# ['det_frac', Effective coverage fraction of each pixel(normalized to 1)
#  'ebv_planck13', E(B-V)
#  'g',
#  'hpix',
#  'i',
#  'r',
#  'star_density_piffv23', star density
#  'z']

In [ ]:
gold['maps']['det_frac'][:].max()

In [ ]:
gold['maps']['det_frac'][:].max()

In [ ]:
maps['g']['sigma_zp'][:][maps['g']['sigma_zp'][:]>0][0:50]

In [ ]:
# ['airmass', airmass 
#  'fwhm', fwhm of psf
#  'mof_depth', no of coadd images
#  'sigma_zp', 
#  'sky_var',
#  'teff_mean', 
#  'teff_sum']

In [ ]:
list(maps['g'].keys())
for key in maps['g'].keys():
    print (key,len(maps['g'][key]))

In [ ]:
for key in maps.keys():
    print ("{:20} has {} objects".format(key,len(maps[key])))

In [ ]:
ipix_maps={}
GOLD_INDEX=np.arange(hp.nside2npix(4096))
for key in ['ebv_planck13','star_density_piffv23','det_frac']:
    hpix=np.full(hp.nside2npix(4096),np.nan)
    hpix[maps['hpix']]=maps[key]
    ipix_maps[key]=np.array(hpix)

In [ ]:
gold_df=pd.DataFrame(data=ipix_maps,index=GOLD_INDEX)

In [ ]:
gold_df[gold_df==hp.UNSEEN]=np.nan

In [ ]:
band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['airmass','fwhm','mof_depth','sigma_zp','sky_var','teff_mean','teff_sum']],
                                     names=['band','data'])

In [ ]:
gold_df_band=pd.DataFrame(index=G_INDEX,columns=band_columns) 

In [ ]:
def get_df_band():
    band_maps={'g':{},'r':{},'i':{},'z':{}}
    for band in ['g','r','i','z']:
        for key in maps['r'].keys():
            downgraded=[]
            hpix=np.full(hp.nside2npix(4096),np.nan)
            hpix[maps['hpix']]=maps[band][key]
            print(hpix)
            downgraded=hp.pixelfunc.ud_grade(hpix, NSIDE, order_in='NESTED', order_out='NESTED',pess=False)
            print(key,downgraded)
            band_maps[band][key]=downgraded
            gold_df_band.loc[:,(band,key)]=downgraded
    return (gold_df_band)

In [ ]:
gold_df_band=get_df_band()

In [ ]:
gold_df_band[gold_df_band==hp.UNSEEN]=np.nan

In [ ]:
# gold_df_band={}
# for band in ['g','r','i','z']:
#     sub_df=pd.DataFrame()
#     for key in maps['r'].keys():
#         hpix=np.full(hp.nside2npix(4096),np.nan)
#         hpix[maps['hpix']]=maps[band][key]
#         downgraded_array=hp.pixelfunc.ud_grade(np.array(hpix), NSIDE, order_in='NESTED', order_out='NESTED',pess=True)
#         sub_df.assign(key,downgraded_array)
#     pd.concat(gold_df_band,axis=1)


# Show star density map

In [ ]:
fig=plt.figure()
hp.mollview(gold_df['star_density_piffv23'].to_numpy(),nest=True,title="Star density of full DES footprint")
hp.graticule()
plt.savefig('/home/cz136/project/balrog/image/hp/star_density.png',dpi=1024)
plt.close()

# Downgrade map

In [ ]:
downgraded_maps={}
for key,value in ipix_maps.items():
    downgraded_maps[key]=hp.pixelfunc.ud_grade(value, NSIDE, order_in='NESTED', order_out='NESTED',pess=False)

In [ ]:
gold_df_downgraded=pd.DataFrame(data=downgraded_maps,index=G_INDEX)

In [ ]:
gold_df_all=pd.concat([gold_df_downgraded,gold_df_band],axis=1,join='inner')

In [ ]:
gold_df_all[gold_df_all==hp.UNSEEN]=np.nan

In [ ]:
gold_df_all.to_pickle('/home/cz136/project/balrog/data/gold_g/gold.pkl')

In [ ]:
gold_join_delta_g_list=[None]*4
for i,catalog in enumerate(ipix_sorted_filtered_grouped):
        gold_join_delta_g_list[i]=pd.concat([catalog,gold_df_all],axis=1,join='inner')
        gold_join_delta_g_list[i].index.name="ipix"

In [ ]:
gng_filled_list=[fill_for_plot(df,['delta_g1','delta_g2']) for df in gold_join_delta_g_list]

In [ ]:
gold_n_delta_g_list=[None]*4
for i,df in enumerate(gng_filled_list):
    gold_n_delta_g_list[i]=pd.concat([gold_df_all,df],axis=1)
    gold_n_delta_g_list[i][gold_n_delta_g_list[1]==hp.UNSEEN]=np.nan

In [ ]:
corr_list=[None]*4
for i,df in enumerate(gold_n_delta_g_list):
    print(len(df))
    corr_list[i]=df.corr(method='pearson').loc[:,['delta_g1','delta_g2']]

In [ ]:
gold_n_delta_g_list[1]

# Correlations (only positive g)

# mcal2: riz w MOF

In [ ]:
corr_list[0].describe()

In [ ]:
corr_list[0]

# mcal3: riz wo MOF

In [ ]:
corr_list[1].describe()

In [ ]:
corr_list[1]

# mcal4: griz w MOF

In [ ]:
corr_list[2].describe()

In [ ]:
corr_list[2]

# mcal5: griz wo MOF

In [ ]:
corr_list[3].describe()

In [ ]:
corr_list[3]

In [ ]:
# fig2,(ax1,ax2)=plt.subplots(ncols=2)

# plt.axes(ax1)
# hp.mollview(gng_filled_list[0]['delta_g1'].to_numpy(),nest=True,title="$\delta g_1$ for riz /w MOF",hold=True)
# hp.graticule()

# plt.axes(ax2)
# hp.mollview(gng_filled_list[0]['star_density_piffv23'].to_numpy(),nest=True,
#             title="Star density plotted on balrog pixels",hold=True)
# hp.graticule()

# # plt.title("Comparing $\delta g$ with other data")

# plt.savefig('/home/cz136/project/balrog/image/hp/compare.png',dpi=2048)
# plt.close()

In [ ]:
delta_g_and_g_filled_list=[fill_for_plot(df,['delta_g1','delta_g2','true_cm_g_1','true_cm_g_2','calculated_g_1','calculated_g_2','ra','dec','bal_id'])
                           for df in gold_join_delta_g_list]

In [ ]:
gold_delta_g_g_list=[None]*4
for i,df in enumerate(delta_g_and_g_filled_list):
    gold_delta_g_g_list[i]=pd.concat([gold_df_all,df],axis=1)

In [ ]:
gold_delta_g_g_list[1].describe()

In [ ]:
for i,df in enumerate(gold_delta_g_g_list):
    path='/home/cz136/project/balrog/data/gold_g/mcal{}.pkl'.format(i+2)
    df.to_pickle(path)

In [ ]:
for i,df in enumerate(final_df_list)